# Mutli Vector Retriever

MultiVectorRetriever 是 LangChain 中一个强大的文档检索工具，它通过为单个文档创建多个向量表示，提高了文档检索的灵活性和准确性。

**核心概念** MultiVectorRetriever 的核心思想是为同一文档生成多个不同的向量表示，每个向量可以捕捉文档的不同方面。这样可以更全面地理解文档的语义信息，从而提高检索的准确性。

**主要特点**
* **多向量生成策略：** MultiVectorRetriever 可以为同一文档生成多个不同的向量表示。
* **灵活性和准确性：** 多个向量表示可以捕捉文档的不同方面，提高检索的灵活性和准确性。

**工作原理**
* **向量生成：** MultiVectorRetriever 首先根据设定的策略（如上述方法），为同一个文档生成多个向量表示。
* **向量存储：** 将生成的多个向量以及它们对应的文档存储起来。
* **检索：** 当用户进行检索时，MultiVectorRetriever 首先将用户的查询转换为向量。然后，它在存储的多个向量中找到与查询向量最相似的向量。
* **返回文档：** 最后，MultiVectorRetriever 返回与找到的相似向量对应的文档。

**优势**
* **更准确的语义匹配：** 通过多个向量表示，可以更全面地捕捉文档的语义信息，提高检索的准确性。
* **更灵活的检索：** 可以根据不同的查询需求，选择不同的向量表示进行检索，提高检索的灵活性。
* **更好的检索效果：** 在处理复杂文档或需要多方面信息检索的场景下，MultiVectorRetriever 可以取得更好的检索效果。

**劣势**
* **索引构建和维护成本高：** MultiVectorRetriever 需要为所有向量构建索引，以便快速检索。对于大规模向量数据集，索引构建和维护成本非常高。此外，当有新的向量加入或删除时，索引需要及时更新，这也会增加维护成本。
* **内存占用高：** MultiVectorRetriever 通常需要将索引加载到内存中，以便快速检索。对于大规模向量数据集，索引的内存占用非常高，这可能会导致内存不足的问题。
* **查询速度受向量维度影响：** MultiVectorRetriever 的查询速度受到向量维度的影响。当向量维度较高时，查询速度可能会变慢。
* **扩展性差：** 当向量数据集规模非常大时，MultiVectorRetriever 可能无法很好地扩展。
* **容错性差：** 如果 MultiVectorRetriever 出现故障，可能会导致检索服务中断，同时纬度较多出现故障概率增加。

In [2]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.storage import InMemoryByteStore
from langchain_ollama import OllamaEmbeddings
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from chat_model_client import get_model

import uuid, pprint

IDKEY = "doc_id"

## 1. 准备文档

In [3]:
loader = WebBaseLoader([
    "https://app.xinhuanet.com/news/article.html?articleId=cf58e389142893239ff9e986e7a3e06a", ## 中国马拉松产业观察：跑出来的消费转型，赛出来的城市活力
    "https://cn.chinadaily.com.cn/a/202412/04/WS67502215a310b59111da71ab.html", ## 算一算马拉松热背后的经济账
    "https://www.cma.gov.cn/2011xwzx/2011xqxxw/2011xzytq/202502/t20250205_6838360.html" ## 强冷空气将影响中东部地区
])
documents = loader.load()
doc_ids = [str(uuid.uuid4()) for _ in documents]

## 2. 构建多维度向量查询器

In [4]:
embeddings = OllamaEmbeddings(model = "llama2-chinese")
## 向量数据库用来存储小块
vectorstore = Chroma(
    collection_name="parent_documents",
    embedding_function=embeddings
)
## store用来存储原始文档
store = InMemoryByteStore()

retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=IDKEY
)

## 3. 子文档分割存储

In [5]:
child_splitter = RecursiveCharacterTextSplitter(chunk_size=500)

child_docs = []
for i, doc in enumerate(documents):
    _id = doc_ids[i]
    _child_docs = child_splitter.split_documents([doc])

    for _doc in _child_docs:
        _doc.type = "child"
        _doc.metadata[IDKEY] = _id
    child_docs.extend(_child_docs)

retriever.vectorstore.add_documents(child_docs)

['8078b362-b367-4dd1-bd2d-c9999706cc49',
 'e16e87d5-a7fd-4fa2-82ac-35f1853fcb48',
 '77720be1-7569-4726-8bdb-01d97654437a',
 'c6dbd2b4-d39e-45d7-bdac-ca3b7ed446dc',
 'a1217cd1-2bca-493d-931f-c5b90e34b9b7',
 '8c86b6f2-9ad7-4186-a698-4705fbde3c8d',
 '4c3a2c21-ac32-4e7e-84ef-5b8059ed0550',
 'ae64530f-e894-46b0-8368-0ba5953e7ae3',
 'de7a965f-02b3-41b7-afe0-afb202d7fb27',
 'b66e3ef6-1b0d-4caf-b69f-d4cad59ac0eb',
 'fed93b35-635e-4df8-8e33-f8f25f366ec4',
 'b90a0e45-80bb-4fd0-bcf0-4bbf8ba5b5de',
 '2e9e2f87-4597-4ab0-bee1-316046785de6',
 'c44e257f-99db-4d33-bda2-34b056de54df',
 '2d092faf-828b-4d8d-8a15-c1b7df360551',
 'ddd69328-4840-428e-bf0a-12680af54f90',
 '35d47059-a5bc-48e8-8cbd-c9e91bd684da',
 'a0ca80a3-a018-45fa-8d2c-88c27bbf720c',
 '7161a169-6d69-476a-9d49-328077fb0da1',
 '8064b1c9-eb19-4024-b044-c9d8623bd88e',
 '3c19f39b-bb15-4a24-9c48-06418fd0bddd',
 '2d5cf93d-ac6e-4991-a537-9c0dc7da8b8c',
 'bd6900bd-334f-490d-899f-4f68b99fb305',
 '0606b309-d0c8-41d2-a395-a59d98899ecd',
 'ba61fb8c-2e7b-

## 4. 文档压缩存储

In [6]:
chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Summarize the following document:\n\n{doc}")
    | get_model('openai')
    | StrOutputParser()
)
summaries = chain.batch(documents, {"max_concurrency": 2})
summary_docs = [
    Document(
        page_content = s,
        metadata={
            "type": "summary",
            IDKEY: doc_ids[i]
        },
    ) for i, s in enumerate(summaries)
]

retriever.vectorstore.add_documents(summary_docs)

/var/folders/5x/c0q41fpx6l540lsl42_bzk5h0000gq/T/ipykernel_95817/2076930780.py:4: UserWarning: Parameters {'presence_penalty', 'top_p', 'frequency_penalty'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  | get_model('openai')


['fe8990fc-1bba-4cd4-ba5a-da9ea5faa395',
 '379a1fb9-4f8e-4886-9e04-b248d4fd2808',
 'f9435f4e-0c6e-4baa-8d33-dce907362ceb']

## 5. 文档提问并存储

In [7]:
from langchain_core.pydantic_v1 import BaseModel, Field

class HypotheticalQuestions(BaseModel):
    questions: list[str] = Field(
        description="An array of hypothetical questions",
        )

chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("Generate three hypothetical questions that the below document could be used to answer:\n\n{doc}")
    # | OpenAI(api_key=os.environ["OPENAI_API_KEY"], base_url=os.environ["OPENAI_BASE_URL"], model='gpt-3.5-turbo', max_retris=0)
    | get_model("openai").bind_tools(tools = [HypotheticalQuestions])
    # | ChatOpenAI(api_key="sk-6211beb174ac4b02944475ba65f93996", base_url=os.environ["DEEPSEEK_BASE_URL"], model='deepseek-chat', max_retries=0).bind_tools(tools=tools)
    # | JsonKeyOutputFunctionsParser(key_name="questions")
    | PydanticToolsParser(tools=[HypotheticalQuestions])
)
# chain.invoke(documents[0])

questions_docs = []
for i, doc in enumerate(documents):
    hypothetical_questions = chain.invoke(doc)
    for q in hypothetical_questions:
        for _q in q.questions:
            questions_docs.append(Document(page_content=_q, metadata={"type": "question", IDKEY: doc_ids[i]}))
retriever.vectorstore.add_documents(questions_docs)

/Users/shaon/anaconda3/envs/wrenv3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
/var/folders/5x/c0q41fpx6l540lsl42_bzk5h0000gq/T/ipykernel_95817/3981831724.py:12: UserWarning: Parameters {'presence_penalty', 'top_p', 'frequency_penalty'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  | get_model("openai").bind_tools(tools =

['66f52af3-3b4d-4f88-9925-59da952aed09',
 'b5fd5ea6-0364-4cae-859c-ecf0dde41a48',
 '5fe8d860-577f-4d75-969a-ff781f500c21',
 '3bd679f7-e391-4bdf-ae13-b8dcb0e752cf',
 'a767cfcd-1806-47af-ba78-b31eb5d04e11',
 'b3783e0b-45cf-4409-86e6-9504dbccffe2',
 'c9182db2-182c-437f-bb29-010bd8a4e8a6',
 '733b2e47-48a3-477c-a105-dad7c9feab0e',
 'b12a67ab-23b2-4284-8e6f-ad2c19f30600']

## 6. 原始文档存储

In [10]:
retriever.docstore.mset(zip(doc_ids, documents))


[Document(metadata={'source': 'https://cn.chinadaily.com.cn/a/202412/04/WS67502215a310b59111da71ab.html', 'title': '算一算马拉松热背后的经济账 - 中国日报网', 'description': '中国日报网', 'language': 'No language found.'}, page_content='\n\n\n\n\n\n算一算马拉松热背后的经济账 - 中国日报网\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n算一算马拉松热背后的经济账\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n站内搜索\n搜狗搜索\n中国搜索\n\n\n\n\n\n\n\n\n\n\n订阅\n移动\nEnglish\n\n\n\n\n\n\n首页\n\n时政\n\n\n\n时政要闻\n\n\n评论频道\n\n\n学习时代\n\n\n两岸频道\n\n\n\n\n资讯\n\n\n\n独家\n\n\n中国日报专稿\n\n\n传媒动态\n\n每日一词\n\n法院公告\n\n\n\n\nC财经\n\n\n\n证券\n\n\n独家\n\n\n科技\n\n\n产业\n\n\n\n\n文旅\n\n\n\n生活\n\n\n中国有约\n\n\n\n视频 \n专栏\n\n\n漫画\n\n观天下\n\n\n中国观察\n中国日报网评\n中国那些事儿\n和评理\n侨一瞧\n\n\n\n地方\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nChina Daily Homepage\n中文网首页\n时政\n资讯\nC财经\n生活\n视频\n专栏\n\n漫画\n原创\n观天下\n地方频道\n\n\n\n\n\n\n\n\n\n\n\n\n\n关闭\n\n\n\n\n\n中国日报网 > 头图 >\n    \n\n\n\n算一算马拉松热背后的经济账\n\n\n\n                来源：新华网\n              \n2024-12-04 

## 7. 检索问题相关的文档

In [ ]:
retriever.get_relevant_documents("马拉松经济价值")